![SAR, InSAR, PolSAR, and banner](https://opensarlab-docs.asf.alaska.edu/opensarlab-notebook-assets/notebook_images/blackboard-banner.png)

# InSAR Time Series Analysis using MintPy and HyP3 products

**Author:** Alex Lewandowski; University of Alaska Fairbanks

Based on the [LosAngeles_time_series](https://github.com/ASFOpenSARlab/opensarlab-notebooks/blob/master/SAR_Training/English/Hazards/LosAngeles_time_series.ipynb) notebook by Eric Fielding, David Bekaert, Heresh Fattahi and Yunjun Zhang, which uses an example ARIA dataset.



## Mapping surface deformation with InSAR time series

This notebook demonstrates how to create an InSAR time series with MintPy using an ASF HyP3 InSAR data stack, which can be ordered on [ASF Data Search/Vertex](https://search.asf.alaska.edu/#/) and prepared using the [Prepare_HyP3_InSAR_Stack_for_MintPy](https://github.com/ASFOpenSARlab/opensarlab-notebooks/blob/master/SAR_Training/English/Master/Prepare_HyP3_InSAR_Stack_for_MintPy.ipynb) notebook.

<img src="https://opensarlab-docs.asf.alaska.edu/opensarlab-notebook-assets/notebook_images/UAFLogo_A_647.png" width="170" align="right" />

It also explores how to assess the quality of the stack inversion, temporal coherence, and velocity errors.

**Important Note about JupyterHub**
Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.

In [ ]:
import url_widget as url_w
notebookUrl = url_w.URLWidget()
display(notebookUrl)

In [ ]:
from IPython.display import Markdown
from IPython.display import display

notebookUrl = notebookUrl.value
user = !echo $JUPYTERHUB_USER
env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/insar_analysis':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "insar_analysis" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select "insar_analysis" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "insar_analysis" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

# 0: Notebook setup

**Import necessary packages**

In [ ]:
import contextlib
import csv
from datetime import datetime, timedelta
from getpass import getpass
import os
from pathlib import Path
import re
from typing import Union
import zipfile

import h5py
from ipyfilechooser import FileChooser
import numpy as np
from osgeo import gdal, osr
from pandas.core.frame import DataFrame
import rasterio
from rasterio.transform import from_origin
from rasterio.warp import transform_bounds, transform
import rioxarray as rxr
from tqdm.notebook import tqdm
import urllib
import xarray as xr

from bokeh.plotting import figure, show, output_file, ColumnDataSource, output_notebook
from bokeh.models import LabelSet

import ipywidgets as widgets
from ipywidgets import Layout

from mintpy.cli import view, tsview, plot_network, plot_transection, plot_coherence_matrix
import mintpy.plot_coherence_matrix
import mintpy.objects.insar_vs_gps
import mintpy.utils

import opensarlab_lib as asfn

**Select the directory holding your MintPy-ready HyP3 data stack and/or MintPy directory from a previously loaded MintPy SBAS stack**
- Click the `Select` button
- Navigate to your data directory
- Click the `Select` button
- Confirm that the desired path appears in green text
- Click the `Change` button to alter your selection

In [ ]:
path = Path.cwd()
fc = FileChooser(path)
display(fc)

**Define a project name and create a MintPy directory in which to store files output during our analysis**

In [ ]:
# define the work directory
work_path = Path(fc.selected_path)
print(f"Work directory: {work_path}")

# define a project name
proj_name = input("Enter a project name: ")

# define the MintPy time-series directory
mint_path = work_path/'MintPy'
mint_path.mkdir(exist_ok=True)
print(f"MintPy directory: {mint_path}")

#create a directory in which to store plots
plot_path = mint_path/"plots"
plot_path.mkdir(exist_ok=True)

# 1. Add Your Climate Data Store (CDS) UID & API Key to the Pyaps3 Config 

### This step only needs to be completed once and may be skipped if you have already updated the CDS config 

- Running the MintPy smallbaselineApp's `correct_troposphere` step requires downloading atmospheric pressure data from the CDS
- If don't yet have a CDS API key:
    - Proceed to [CDS](https://cds.climate.copernicus.eu/cdsapp#!/home) and create an account
    - Open the [Datasets page](https://cds.climate.copernicus.eu/cdsapp#!/search?type=dataset)
    - Search for "ERA5"
    - Select any of ERA5 datasets that appear
    - Select the `Download data` tab
    - Scroll to the bottom of the screen
    - Accept the `Terms of use`
    - Click on your name at the top right of the screen to access your profile page
    - Your `UID` and `API Key` will be displayed here 
- Run the following 2 code cells to update the pyaps config from this notebook **OR** open an OpenSARlab terminal and complete the following steps:
    - Use vim or another text editor to open the `~/.cdsapirc` config file
        - Add the CDS url to the first line of the config and your CDS `UID` and CDS`API Key` to the 2nd line of the config
            - This should be formatted like:
                - url: https://cds.climate.copernicus.eu/api/v2\
                - key: your_UID: your_API_Key
        - Save the config and exit the text editor

**If you do not add your CDS credentials to `~/.cdsapirc`, the `correct_troposphere` step will fail**

In [ ]:
pyaps_cfg = Path("/home/jovyan/.cdsapirc")
try:
    with open(pyaps_cfg, 'r') as f:
        lines = f.readlines()
        if len(lines) == 2 and 'url' in lines[0]:
            print("There was a CDS UID and API Key found in the pyaps3 config: ~/.cdsapirc")
            print("Would you like to update them?")
            update_cds_cfg = asfn.select_parameter(["Do not update CDS UID and API Key", 
                                                    "Update CDS UID and API Key"])
            display(update_cds_cfg)
        else:
            update_cds_cfg = None
except FileNotFoundError:
    update_cds_cfg = None

In [ ]:
if not update_cds_cfg or "Update" in update_cds_cfg.value:
    with open(pyaps_cfg, 'w') as f:
            uid = input("Enter your CDS UID")
            key = getpass("Enter your CDS API Key")
            lines = ['', '']
            lines[0] = f"url: https://cds.climate.copernicus.eu/api/v2\n"
            lines[1] = f"key: {uid}:{key}\n"
            f.seek(0)
            f.writelines(lines)
            f.truncate()

# 2. smallbaselineApp.py overview

This application provides a workflow which includes several steps to invert a stack of unwrapped interferograms and apply different corrections to obtain ground displacement timeseries.  
The workflow consists of two main blocks:

* correcting unwrapping errors and inverting for the raw phase time-series (blue ovals),
* correcting for noise from different sources to obtain the displacement time-series (green ovals).

Some steps are optional, which are switched off by default (marked by dashed boundaries). Configuration parameters for each step are initiated with default values in a customizable text file: [smallbaselineApp.cfg](https://github.com/insarlab/MintPy/blob/master/mintpy/defaults/smallbaselineApp.cfg). In this notebook, we will walk through some of these steps, for a complete example see the [MintPy repository](https://github.com/insarlab/MintPy).


<p align="left">
  <img width="600" src="https://opensarlab-docs.asf.alaska.edu/opensarlab-notebook-assets/notebook_images/MintPyWorkflow.jpg">
</p>     
<p style="text-align: center;">
    (Figure from Yunjun et al., 2019)
</p>

## 2.1. Processing steps of smallbaselineApp.py

The MintPy **smallbaselineApp.py** application provides a workflow to invert a stack of unwrapped interferograms and apply different (often optional) corrections to obtain ground displacement timeseries. A detailed overview of the options can be retrieved by involking the help option:

In [ ]:
!smallbaselineApp.py --help

## 2.2. Configuring processing parameters

The processing parameters for the **smallbaselineApp.py** are controlled through a configuration file. If no file is provided the default [smallbaselineApp.cfg](https://github.com/insarlab/MintPy/blob/master/mintpy/defaults/smallbaselineApp.cfg) configuration is used. We will create a custom config file with modified configuration parameters for this time-series analysis. Any options added to the custom config will override options set in the default config.

In [ ]:
print("Do you wish to skip the template creation and data loading steps to work with a previously loaded dataset?")

skip_load_choice = asfn.select_parameter(['create a MintPy template and load data', 'skip template creation and data loading steps'])
display(skip_load_choice)

In [ ]:
load = 'skip' not in skip_load_choice.value

if load:
    config = f'''# vim: set filetype=cfg:
    mintpy.load.processor        = hyp3
    ##---------interferogram datasets:
    mintpy.load.unwFile          = {work_path}/*/*unw_phase_clip.tif
    mintpy.load.corFile          = {work_path}/*/*corr_clip.tif
    ##---------geometry datasets:
    mintpy.load.demFile          = {work_path}/*/*dem_clip.tif
    mintpy.load.incAngleFile     = {work_path}/*/*lv_theta_clip.tif
    mintpy.load.azAngleFile      = {work_path}/*/*lv_phi_clip.tif
    mintpy.load.waterMaskFile    = {work_path}/*/*water_mask_clip.tif
    '''

In [ ]:
if load:
    ref_point_option = asfn.select_parameter(["Allow MintPy to determine a reference point", 
                                             "Define a reference point"])
    display(ref_point_option)

In [ ]:
if load:
    is_float = False
    ref_pt_config = ''
    while not is_float:
        if 'Define' in ref_point_option.value:
            try:
                lat = float(input("Enter reference latitude"))
                lon = float(input("Enter reference longitude"))
                is_float = True
            except ValueError:
                print("Latitude and Longitude must be convertable to float")
                continue
            ref_pt_config = f'\nmintpy.reference.lalo        = {lat},{lon}'
        else:
            break

In [ ]:
if load:
    ref_date_option = asfn.select_parameter(["Allow MintPy to determine reference date", 
                                             "Reference time-series to earliest date in stack"])

    display(ref_date_option)

In [ ]:
if load:
    ref_date_config = ''
    if 'MintPy' in ref_date_option.value:
        ref_date_config = f'\nmintpy.reference.date       = auto'
    else:
        ref_date_config = f'\nmintpy.reference.date       = no'

**MintPy allows for multithreaded processing using Dask for interferogram inversion and topographic correction.**

MintPy defaults to sequential processing. 
In general, multithreaded processing will be faster. 

If you opt to use multithreaded proccessing, be aware that Dask will use CPU cores on your compute instance, which may impact the processing speeds for scripts and other notebooks running at the same time.

https://www.dask.org/

Note: The following code cell may list more CPU cores than available if you are running this on a shared instance in a Jupyter Hub, such as OpenSARLab. In this case, if opting to use all available cores for Dask multi-threading, multiple Dask workers will be assigned to each CPU core. The code will still run and you will still see a speed improvement compared to sequential processing.  

In [ ]:
if load:
    cpu_count = os.cpu_count()

    print(f"You currently have access to {cpu_count} logical CPU cores")
    multithread_option = asfn.select_parameter(["Do not use multithreaded processing",
                                         f"Use all {cpu_count} available cores for multithreaded processing",
                                         "Use some of my available cores for multithreaded processing"])
    display(multithread_option)

In [ ]:
if load:
    if 'Use all' in multithread_option.value:
        multithread_config = f'''\nmintpy.compute.cluster      = local
    mintpy.compute.numWorker    = {cpu_count}'''
    elif 'Use some' in multithread_option.value:
        valid_cpu_count = False
        worker_cores = 9999999999
        while not valid_cpu_count:
            try:
                worker_cores = int(input(f"Enter the number of CPU cores to use for multithreading (must be {cpu_count} or less)"))
            except ValueError:
                pass
            valid_cpu_count = worker_cores <= cpu_count
        multithread_config = f'''\nmintpy.compute.cluster      = local
    mintpy.compute.numWorker    = {worker_cores}'''
    else:
        multithread_config = ''    

In [ ]:
config_path = mint_path/f'{proj_name}.txt'

if load:
    updated_config = f'{config}{ref_pt_config}{ref_date_config}{multithread_config}'

    with open(config_path, 'w') as f:
        f.write(updated_config)
        print(f"config file path: {config_path}\n")

    with open(config_path, 'r') as f:
        for line in f.readlines():
            print(line)

# 3. Small Baseline Time Series Analysis

**You can run every step in smallbaselineApp.py with one call, using the command in the cell below**

**For the purposes of this tutorial, we will run each step separately**

We will run the steps:
- load_data
- modify_network
- reference_point
- quick_overview
- invert_network
- correct_troposphere
- correct_topography
- residual_RMS
- reference_date
- velocity
- google_earth

Skipped steps include:
- correct_unwrap_error
- correct_LOD
- correct_SET
- deramp
- hdfeos5

Skipped steps will also be skipped if running the entire smallbaselineApp in the cell below.

In [ ]:
# This runs every step

# !smallbaselineApp.py --work-dir {mint_path}  {config_path}

## 3.1. Load Data

**Run the `load_data` step**

- If you get a missing 'Height' attribute error, you are missing a DEM, which is an available option when ordering HyP3 InSAR products


In [ ]:
if load:
    !smallbaselineApp.py $config_path --work-dir {mint_path} --dostep load_data

The output of the loading step is an "inputs" directory containing two HDF5 files:
- ifgramStack.h5: This file contains 6 dataset cubes (e.g. unwrapped phase, coherence, connected components etc.) and multiple metadata.  
- geometryGeo.h5: This file contains geometrical datasets (e.g., incidence/azimuth angle, masks, etc.). 

In [ ]:
inputs_path = mint_path/'inputs'
!ls $inputs_path

<div class="alert alert-info">
<b>info.py :</b> 
To get general infomation about a MintPy product, run info.py on the file.   
</div>

## 3.2. Modify the Network

**Run the `modify_network` step**

- Identifies and excludes interferograms (i.e. affected by remaining coherence phase-unwrapping errors) before the network inversion

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep modify_network

## 3.3. Plot the interferogram network

Running **plot_network.py** gives an overview of the network and the average coherence of the stack. The program creates multiple files as follows:
- `ifgramStack_coherence_spatialAvg.txt`: Contains interferogram dates, average coherence temporal and spatial baseline separation.
- `Network.pdf`: Displays the network of interferograms on time-baseline coordinates, colorcoded by avergae coherence of the interferograms. 
- `CoherenceMatrix.pdf` shows the avergae coherence pairs between all available pairs in the stack.

In [ ]:
%matplotlib inline
with asfn.work_dir(mint_path):
    plot_network.main([f'{inputs_path}/ifgramStack.h5'])
    plots = ['bperpHistory.pdf', 'coherenceHistory.pdf', 'coherenceMatrix.pdf', 'network.pdf']
    for p in plots:
        if (mint_path/p).exists():
            (mint_path/p).rename(f'{plot_path}/{p}')

## 3.4. Set the Reference Point

**Run the `reference_point` step**

The interferometric phase is a relative observation by nature. The phases of each unwrapped interferogram are relative with respect to an arbitrary pixel. Therfore, we need to reference all interferograms to a common reference pixel.

The `reference_point` step selects a common reference pixel for the stack of interferograms. The default approach of MintPy is to choose a pixel with the highest spatial coherence in the stack. Other options include specifying the longitude and latitude of the desired reference pixel or the line and column number of the refence pixel.   

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep reference_point

**Running the "reference_step" adds additional attributes "REF_X, REF_Y" and "REF_LON, REF_LAT" to the `ifgramStack.h5` file. To see the attributes of the file run `info.py`**

In [ ]:
!info.py $inputs_path/ifgramStack.h5 | egrep 'REF_'

## 3.5. Run a Quick Overview

**Run the `quick_overview` step**

- Assess possible groud deformation using the velocity from traditional interferogram stacking 
    - *reference: Zebker et al. (1997, JGR)*
- Assess distribution of phase unwrapping error from the number of interferogram triplets with non-zero integer ambiguity of closure phase 
    - *reference: T_int in Yunjun et al. (2019, CAGEO). Related to section 3.2, equation (8-9) and Fig. 3d-e.*

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep quick_overview

## 3.6. Inverting the Small Baseline network

**Run the `invert_network` step**

- Invert the network of differential unwrapped interferograms to estimate the time-series of unwrapped phase with respect to a reference acquisition date
- By default mintpy selects the first acquisition
- The estimated time-series is converted to distance change from radar to target and is provided in meters. 

In [ ]:
ifgram_path = mint_path/"inputs/ifgramStack.h5"

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep invert_network

## 3.7. Correct for Tropospheric Propagation Delays (Optional)

**Run the `correct_troposphere` step**

- Uses ECMWF [ERA5 climate reanalysis pressure data](https://cds.climate.copernicus.eu/cdsapp#!/search?type=dataset&keywords=((%20%22Product%20type:%20Reanalysis%22%20)%20AND%20(%20%22Provider:%20Copernicus%20C3S%22%20))&text=pressure)
- CDS limits ECMWF archive requests to 50, so your requests may be queued until there is space.
    - https://cds.climate.copernicus.eu/live/queue

In [ ]:
if load:
    tropo_choice = asfn.select_parameter(["Perform Tropospheric Correction Step",
                                          "Skip Tropospheric Correction Step",
                                          "Delete Outputs of a Previous (possibly interrupted) Troposheric Correction and Rerun",
                                          "Delete Outputs of a Previous Troposheric Correction and Skip Troposheric Correction Now"])
    display(tropo_choice)
else:
    tropo_files = list(mint_path.glob('*ERA5*.h5'))
    if 0 < len(tropo_files) < 3:
        # some but not all tropo corrected HDF5s are present and we should delete outputs rerun troposheric correction
        tropo_choice = "Delete and Rerun"
    elif len(tropo_files) == 0:
        # tropospheric correction was not performed
        tropo_choice = "Skip"
    else:
        # tropospheric correction was performed and all necessary HDF5 files are present
        tropo_choice = "Done"

In [ ]:
def set_troposhperic_correction_mintpy(config_path, method):
    config_path = Path(config_path)
    with open(config_path, 'r') as f:
        config = f.readlines()    
    config_update = config
    
    for i, l in enumerate(config):
        no_comment = l.split("#")[0]
        present = False
        if "mintpy.troposphericDelay.method=" in "".join(no_comment.split()):
            config_update[i] = f"mintpy.troposphericDelay.method = {method}"
            present = True
            break     
    if not present:
        config_update.append(f"\nmintpy.troposphericDelay.method = {method}")

    config_str = ""
    for l in config_update:
        config_str = f"{config_str}{l}"
    
    with open(config_path, 'w') as f:
        f.write(config_str)

In [ ]:
if type(tropo_choice) != str:
    tropo_choice = tropo_choice.value

correct_tropo = "Perform" in tropo_choice or "Rerun" in tropo_choice or "Done" in tropo_choice

era5_path = mint_path/"ERA5"
timeseries_era5_path = mint_path/"timeseries_ERA5.h5"
inputs_era5_path = mint_path/"inputs/ERA5.h5"

if "Delete" in tropo_choice:
    for f in [timeseries_era5_path, inputs_era5_path]:
        try:
            f.unlink()
        except FileNotFoundError:
            pass
    try:
        shutil.rmtree(era5_path)
    except FileNotFoundError:
        pass

if correct_tropo and "Done" not in tropo_choice:
    set_troposhperic_correction_mintpy(config_path, "pyaps")
    !smallbaselineApp.py $config_path --work-dir {mint_path} --dostep load_data
    !smallbaselineApp.py $config_path --work-dir {mint_path} --dostep correct_troposphere
elif not correct_tropo:
    set_troposhperic_correction_mintpy(config_path, "no")
    !smallbaselineApp.py $config_path --work-dir {mint_path} --dostep load_data

## 3.8. Correct for DEM Errors

**Run the `correct_topography` step**

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep correct_topography

## 3.9. Calculate the Root Mean Square (RMS) of Residual Phase Time-Series for Each Acquisition

**Run the `residual_RMS` step**

- *reference: Yunjun et al. (2019, section 4.9 and 5.4)*
- To remove the long wavelength component in space, a phase ramp is removed for each acquisition
- Sets optimal reference date to date with min RMS
- Sets exclude dates (outliers) to dates with RMS > cutoff * median RMS (Median Absolute Deviation)

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep residual_RMS

## 3.10. Reference the Entire Time-Series to One Date in Time

**Run the `reference_date` step**

- *reference: Yunjun et al. (2019, section 4.9)*

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep reference_date

## 3.11. Estimate The Long-Term Velocity Rate

**Run the `velocity` step**

The timeseries file contains three datasets:
- the `time-series` dataset, which is the interferometric range change for each acquisition relative to the reference acquisition
- the `date` dataset, which contains the acquisition date for each acquisition
- the `bperp` dataset, which contains the timeseries of the perpendicular baseline 

The ground deformation caused by many geophysical or anthropogenic processes are linear at first order approximation. Therefore it is common to estimate the rate of the ground deformation which is the slope of linear fit to the time-series. 

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep velocity

In [ ]:
scp_args = f'{mint_path}/velocity.h5 velocity -v -1 40 --dpi 600 --figsize 15 15 --outfile {plot_path}/velocity.png'
view.main(scp_args.split())

<div class="alert alert-info">
<b>Note :</b> 
Negative values indicates that target is moving away from the radar (i.e., Subsidence in case of vertical deformation).
Positive values indicates that target is moving towards the radar (i.e., uplift in case of vertical deformation). 
</div>

## 3.12. Geocode velocity.h5 in Preparation for Creating a velocity.kmz

**Run the `geocode` step**

- This is unnecessary for geocoded HyP3 data but would be needed for non-geocoded data

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep geocode

## 3.13. Create a kmz File

**Run the `google_earth` step**

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep google_earth

## 3.14 Plot the unwrapped inverted timeseries

**Create directories in which to store output we will create in upcoming steps**

In [ ]:
geotiff_path = mint_path/'GeoTiffs'
geotiff_path.mkdir(exist_ok=True)

disp_path = geotiff_path/'displacement_maps'
disp_path.mkdir(exist_ok=True)

wrapped_path = disp_path/"wrapped"
wrapped_path.mkdir(exist_ok=True)

unwrapped_path = disp_path/"unwrapped"
unwrapped_path.mkdir(exist_ok=True)

demErr = 'timeseries_ERA5_demErr.h5' if correct_tropo else 'timeseries_demErr.h5'
ts_demErr = mint_path/f'{demErr}'

**Plot the unwrapped inverted time series steps (n, n+1, n+2, etc...)**

- save a png

In [ ]:
scp_args = f'{ts_demErr} --notitle --notick --noaxis --dpi 600 --figsize 15 15 --outfile {unwrapped_path}/unwrapped_inverted_ts.png'
view.main(scp_args.split())

# 4. Error analysis (signal vs noise)

Uncertainty of the ground displacement products derived from InSAR time-series, depends on the quality of the inversion of the stack of interferograms and the accuracy in separating the ground displacement from other components of the InSAR data. Therefore the definition of signal vs noise is different at the two main steps in MintPy:  

1) During the inversion: 
    At this step all systematic components of the interferometric phase (e.g., ground displacement, propagation delay, geometrical residuals caused by DEM or platform's orbit inaccuracy) are considered signal, while the interferometric phase decorrelation, phase unwrapping error and phase inconsistency are considered noise. 
    
2) After inversion: the ground displacement component of the time-serieses is signal, and everything else (including the propagation delay and geometrical residuals) are considered noise

Therefore we first discuss the possible sources of error during the inversion and the existing ways in MintPy to evaluate the quality of inversion and to improve the uncertainty of the inversion. Afterwards we explain the different components of the time-series and the different processing steps in MintPy to separate them from ground displacement signal.  


## 4.1. Quality of the inversion

The main sources of noise during the time-series inversion includes decorrelation, phase unwrapping error and the inconsistency of triplets of interferofgrams. Here we mainly focus on the decorrelation and unwrapping errors. We first show the existing quantities in MintPy to evaluate decorrelation and unwrapping errors and then discuss the existing ways in MintPy to reduce the decorrelation and unwrapping errors on the time-series inversion.

### 4.1.1. Average spatial coherence

Mintpy computes temporal average of spatial coherence of the entire stack as a potential ancillary measure to choose reliable pixels after time-series inversion.

In [ ]:
%matplotlib inline
scp_args = f"{mint_path}/avgSpatialCoh.h5 --dpi 600 --figsize 15 15 --outfile {plot_path}/avg_spatial_coh.png"
view.main(scp_args.split())

### 4.1.2. Temporal coherence

In addition to timeseries.h5 which contains the time-series dataset, invert_network produces other quantities, which contain metrics to evaluate the quality of the inversion including temporalCoherence.h5. Temporal coherence represents the consistency of the timeseries with the network of interferograms. 

Temporal coherence varies from 0 to 1. Pixels with values closer to 1 are considered reliable and pixels with values closer to zero are considered unreliable. For a dense network of interferograms, a threshold of 0.7 may be used (Yunjun et al, 2019).

In [ ]:
%matplotlib inline
scp_args = f"{mint_path}/temporalCoherence.h5 --dpi 600 --figsize 15 15 --outfile {plot_path}/temporal_coh.png"
view.main(scp_args.split())

## 4.2. Velocity error analysis

The estimated velocity also comes with an expression of unecrtainty which is simply based on the goodness of fit while fitting a linear model to the time-series. This quantity is saved in "velocity.h5" under the velocityStd dataset. 

In [ ]:
%matplotlib inline
scp_args = f'{mint_path}/velocity.h5 velocityStd -v 0 1 --dpi 600 --figsize 15 15 --outfile {plot_path}/velocity_err.png'
view.main(scp_args.split())

Note that the plot above is the velocity error, not the velocity. The errors generally increases with distance from the reference point and can also increase for points with elevations different from the reference point if topographically correlated water vapor variations are especially strong in the area.

## 4.3. Compare InSAR time-series with GPS time-series in LOS direction

- http://geodesy.unr.edu/NGLStationPages/gpsnetmap/GPSNetMap.html
- http://geodesy.unr.edu/NGLStationPages/DataHoldings.txt

### 4.3.1. Identify Potential GPS stations

**Write the University of Nevada, Reno GPS station holdings metadata to GPS_stations.csv**

In [ ]:
with asfn.work_dir(mint_path):
    url = 'http://geodesy.unr.edu/NGLStationPages/DataHoldings.txt'
    response = urllib.request.urlopen(url, timeout=5)
    content = response.read()
    rows = content.decode('utf-8').splitlines()
    holdings_txt = Path('.')/'DataHoldings.txt'
    if holdings_txt.exists():
        holdings_txt.unlink()

with open(f'{mint_path}/GPS_stations.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',', escapechar=',', quoting=csv.QUOTE_NONE)
    for row in rows:
        csv_writer.writerow([re.sub('\s+', ' ', row)])


**Build a list of GPS stations within your area of interest**

In [ ]:
def convert_long(long):
    if long > 180:
        long = long - 360
    return long

# get the InSAR stack's corner coordinates
with h5py.File(f"{mint_path}/inputs/geometryGeo.h5", 'r') as f:
    lon_west = float(f.attrs['LON_REF1'])
    lon_east = float(f.attrs['LON_REF2'])
    lat_south = float(f.attrs['LAT_REF1'])
    lat_north = float(f.attrs['LAT_REF3'])

# get the start and end dates of the timeseries
if correct_tropo:
    info = gdal.Info(f"{mint_path}/timeseries_ERA5_demErr.h5", format='json')
else:
    info = gdal.Info(f"{mint_path}/timeseries_demErr.h5", format='json')
ts_start = info['metadata']['']['START_DATE']
ts_start = datetime.strptime(ts_start, '%Y%m%d')
ts_end = info['metadata']['']['END_DATE']
ts_end = datetime.strptime(ts_end, '%Y%m%d')

# find all stations that have data within the ts time range,
# are located within the AOI and at an unmasked pixel location
gps_stations = list()
with open(f'{mint_path}/GPS_stations.csv', newline='') as csvfile:
    csv_reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in list(csv_reader)[1:]:
        begin_date = datetime.strptime(row[7], '%Y-%m-%d')
        mod_date = datetime.strptime(row[9], '%Y-%m-%d')
        lat = float(row[1])
        
        lon = convert_long(float(row[2]))

        n = [lat, lon]
        a = [lat_north, lon_west]
        b = [lat_south, lon_west]
        c = [lat_south, lon_east]
        ab = np.subtract(a, b)
        an = np.subtract(a, n)
        bc = np.subtract(b, c)
        bn = np.subtract(b, n)

        in_aoi = 0 <= np.dot(ab, an) <= np.dot(ab, ab) and 0 <= np.dot(bc, bn) <= np.dot(bc, bc)
        in_date_range = ts_start >= begin_date and ts_end <= mod_date
        
        if in_aoi and in_date_range:
            vel_file = f'{mint_path}/velocity.h5'
            atr = mintpy.utils.readfile.read_attribute(vel_file)
            coord = mintpy.utils.utils.coordinate(atr, lookup_file=f'{mint_path}/inputs/geometryRadar.h5')
            y, x = coord.geo2radar(lat, lon)[:2]
            msk = mintpy.utils.readfile.read(f'{mint_path}/maskTempCoh.h5')[0]
            box = (x, y, x+1, y+1)
            masked = not msk[y, x]
            if not masked:
                gps_stations.append(row[0].strip())
                
gps = len(gps_stations) > 0
if not gps:
    print("There were no GPS sites found in your AOI")

**Create a dictionary of metadata for each GPS site in yoour AOI**

In [ ]:
if gps:
    def get_gps_dict(stations):
        gps_dict = {}
        with open(f'{mint_path}/GPS_stations.csv', newline='') as csvfile:
            csv_reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
            for row in list(csv_reader)[1:]:
                if row[0] in stations:
                    gps_dict[row[0]] = {
                        'lat': row[1],
                        'long': row[2],
                        'height':row[3],
                        'x': row[4],
                        'y': row[5],
                        'z': row[6],
                        'date_beg': row[7],
                        'date_end': row[8],
                        'date_mod': row[9],
                        'num_sol': row[10],
                        'st_og_name': 'na'
                    }
                    if len(row) > 11:
                         gps_dict[row[0]]['st_og_name'] = row[11]
        return gps_dict
    
    gps_dict = get_gps_dict(gps_stations)

    _, my_dict = mintpy.utils.readfile.read(f'{mint_path}/inputs/geometryGeo.h5', datasetName='height')

    x_first = float(my_dict['X_FIRST'])
    x_step = float(my_dict['X_STEP'])
    width = float(my_dict['WIDTH'])

    y_first = float(my_dict['Y_FIRST'])
    y_step = float(my_dict['Y_STEP'])
    height = float(my_dict['LENGTH'])

    # (xmin, ymin, xmax, ymax)
    bounds = (x_first, y_first+(y_step*height), x_first+(x_step*width),  y_first)

    # convert bounds to web-mercator (epsg:3857)
    xmin, ymin, xmax, ymax = transform_bounds(4326, 3857, *bounds)

**Plot the GPS station locations in your AOI**

Hover over GPS sites to view site metadata

In [ ]:
if gps:
    output_notebook()

    longs = [convert_long(float(gps_dict[k]['long'])) for k in gps_stations]
    lats = [float(gps_dict[k]['lat']) for k in gps_stations]
    xy_web = transform(4326, 3857, longs, lats)

    source = DataFrame(
        data=dict(
            x=xy_web[0],
            y=xy_web[1],
            stations=gps_dict.keys(),
            lats=[gps_dict[k]['lat'] for k in gps_dict],
            longs=[convert_long(float(gps_dict[k]['long'])) for k in gps_dict],
            exes=[gps_dict[k]['x'] for k in gps_dict],
            whys=[gps_dict[k]['y'] for k in gps_dict],
            zees=[gps_dict[k]['z'] for k in gps_dict],
            heights=[gps_dict[k]['height'] for k in gps_dict],
            start_dates=[gps_dict[k]['date_beg'] for k in gps_dict],
            end_dates=[gps_dict[k]['date_end'] for k in gps_dict],
            mod_dates=[gps_dict[k]['date_mod'] for k in gps_dict],
            num_sols=[gps_dict[k]['num_sol'] for k in gps_dict],
            og_names=[gps_dict[k]['st_og_name'] for k in gps_dict],
        )
    )

    labels = LabelSet(
                x='x',
                y='y',
                text='stations',
                level='glyph',
                x_offset=-15, 
                y_offset=15, 
                source=ColumnDataSource(source))

    TOOLTIPS = [
        ("station", "@stations"),
        ("lat(deg)", "@lats"),
        ("long(deg)", '@longs'),
        ('height(m)', '@heights'), 
        ("x(m)", '@exes'),
        ("y(m)", "@whys"),
        ("z(m)", "@zees"),
        ("start date", "@start_dates"),
        ("end date", "@end_dates"),
        ("modification date", "@mod_dates"),
        ("NumSol", "@num_sols"),
        ("original station name", "@og_names")

    ]

    # range bounds supplied in web mercator coordinates
    p = figure(x_range=(xmin, xmax), y_range=(ymin, ymax),
               x_axis_type="mercator", y_axis_type="mercator", tooltips=TOOLTIPS)

    p.add_tile("STAMEN_TERRAIN_RETINA")

    p.circle_dot(x='x', y='y', size=20, fill_alpha=0.2, color='red', alpha=0.6, source=source)

    p.add_layout(labels)
    
    p.title = "GPS Site Locations"

    show(p)

**Select a GPS station**

In [ ]:
if gps:
    gps_station = widgets.RadioButtons(
        options=gps_stations,
        description='',
        disabled=False,
        layout=Layout(min_width='800px'))
    
    print("Select a GPS station")
    display(gps_station)

**Plot the GPS/velocity comparison**

In [ ]:
%matplotlib widget
if gps:
    scp_args = f"{mint_path}/velocity.h5 velocity --show-gps --ref-gps {gps_station.value} --gps-comp enu2los --gps-label --figsize 9 9"
    with asfn.work_dir(mint_path):
        view.main(scp_args.split())

# 5. Plotting a Motion Transect 

**Select the transect to plot**

In [ ]:
%matplotlib widget
data, _ = mintpy.utils.readfile.read(ts_demErr, datasetName='timeseries')
mask = np.ma.masked_where(data[11]==0, data[11])
data = mask.filled(fill_value=np.nan)
line = asfn.LineSelector(data, figsize=(9, 9), cmap='jet')

In [ ]:
amp = list(work_path.glob('*/*_amp_clip.tif'))[0]
amp = gdal.Open(str(amp))
geotrans = amp.GetGeoTransform()

def geolocation(x, y, geotrans):
    return [geotrans[0]+x*geotrans[1], geotrans[3]+y*geotrans[5]]

try:
    pnt_1 = geolocation(line.pnt1[0][0], line.pnt1[0][1], geotrans)
    pnt_2 = geolocation(line.pnt2[0][0], line.pnt2[0][1], geotrans)
    print(f"point 1: {pnt_1}")
    print(f"point 2: {pnt_2}")
except TypeError:
    print('TypeError')
    display(Markdown(f'<text style=color:red>This error may occur if a line was not selected.</text>'))

In [ ]:
%matplotlib inline
scp_args = f'{mint_path}/velocity.h5 --start-lalo {pnt_1[1]} {pnt_1[0]} --end-lalo {pnt_2[1]} {pnt_2[0]} --outfile x'
with asfn.work_dir(plot_path):
    plot_transection.main(scp_args.split())

# 6. Plot the Cumulative Displacement Map and Point Displacement Time Series

- Use the `Time` bar below the Cumulative Displacement Map to view displacements for different time periods
- Click on the Cumulative Displacement Map to select points for displaying Point Displacement Time-Series

In [ ]:
%matplotlib widget

tsview.main([str(ts_demErr), 
                    f'-d={mint_path}/inputs/geometryGeo.h5', 
                    f'-o={mint_path}/displacement_ts', 
                    f'--outfile={mint_path}/displacement_ts.pdf'])

# 7. Generate coherence, velocity, and total displacement Geotiffs

**Create a list dates for all timesteps**

In [ ]:
ifgramstack = inputs_path/"ifgramStack.h5"

with h5py.File(ifgramstack, "r") as f:
    dates = f["date"][()]
    dates = list(set([d.decode("utf-8") for insar in dates for d in insar]))
    dates.sort()
dates

**Save the full displacement timeseries**

In [ ]:
ds = f'{dates[0]}_{dates[-1]}'
!save_gdal.py $ts_demErr -d $ds --of GTIFF -o $geotiff_path/"save_gdal_ts_demErr.tif"

**Save the unwrapped displacement GeoTiffs**

In [ ]:
with h5py.File(ifgramstack, 'r') as f:
    unw_pth = f.attrs['FILE_PATH']

ds_unw = rasterio.open(unw_pth, 'r', driver='GTiff')

for i, d in enumerate(tqdm(dates)):
    date_range = f'{dates[0]}_{dates[i]}'
    cmd = f'view.py {ts_demErr} {date_range} --notitle --notick --noaxis'
    data, _, _ = mintpy.view.prep_slice(cmd)
    data = data / 100 # cm -> meters
        

    with rasterio.open(f'{unwrapped_path}/{date_range}_{ts_demErr.stem}_unwrapped.tif', 'w', driver='GTiff',
                  height = data.shape[0], width = data.shape[1],
                  count=1, dtype=str(data.dtype),
                  crs=ds_unw.read_crs(),
                  transform=ds_unw.transform,
                  nodata=np.nan) as ds:
        
        ds.write(data.astype(rasterio.float32), 1)

**Write a function to add a color ramp to single band GeoTiff**

In [ ]:
def colorize_wrapped_insar(tif_path: Union[str, Path]):
    """
    Blue: 0 and 2π
    Red: π/2
    Yellow: π
    Green 3/2π
    """
    ds = gdal.Open(str(tif_path), 1)
    band = ds.GetRasterBand(1)

    # create color table
    colors = gdal.ColorTable()
    
    colors.CreateColorRamp(0, (0, 0, 255),  64, (255, 0, 0)) 
    colors.CreateColorRamp(64, (255, 0, 0),   128, (255, 255, 0))
    colors.CreateColorRamp(128, (255, 255, 0), 192, (0, 255, 0))
    colors.CreateColorRamp(192, (0, 255, 0),   255, (0, 0, 255))

    # set color table and color interpretation
    band.SetRasterColorTable(colors)
    band.SetRasterColorInterpretation(gdal.GCI_PaletteIndex)

    # close and save file
    del band, ds

**Collect paths to unwrapped displacement maps**

In [ ]:
unwrapped_paths = list(unwrapped_path.rglob('*_unwrapped.tif'))
unwrapped_paths

**Generate the wrapped interferogram GeoTiffs**

- Please note that the wrapped range used below is currently under review and may not yet correctly correspond to the Sentinel-1 wavelength 

In [ ]:
sentinel_c_band_lambda = 5.5465763

for unw_path in tqdm(unwrapped_paths):
    date_range_regex = '(?<=/unwrapped/)\d{8}_\d{8}'
    date_range = re.search(date_range_regex, str(unw_path)).group(0)
    
    with rxr.open_rasterio(unw_path, masked=True).squeeze() as ds:
        # convert unwrapped raster to radians
        with xr.set_options(keep_attrs=True):
            unw_rad = (ds * -4 * np.pi) / sentinel_c_band_lambda
          
    # I don't know what it means to convert meters to radians
    # since we did that to the unw data, let's try doing the same to the wrapped range
    wrap_range = [
        (-2.8 * -4 * np.pi) / (sentinel_c_band_lambda * 100),
        (2.8 * -4 * np.pi) / (sentinel_c_band_lambda * 100)
    ]
       
    # wrap the interferogram
    with xr.set_options(keep_attrs=True):
        wrap = mintpy.utils.utils0.wrap(unw_rad, wrap_range=wrap_range)
        
    # collect crs and transform
    with rasterio.open(unw_path, 'r', driver='GTiff') as ds:
        unw_crs = ds.read_crs()
        unw_transform = ds.transform
    
    # Save wrapped interferogram as a GeoTiff
    wrp_path = wrapped_path/f'{date_range}_{ts_demErr.stem}_wrapped_unscaled.tif'
    with rasterio.open(wrp_path, 'w', driver='GTiff',
                      height = wrap.shape[0], width = wrap.shape[1],
                      count=1, dtype=str(wrap.dtype),
                      crs=unw_crs,
                      transform=unw_transform,
                      nodata=np.nan) as ds:
        ds.write(wrap.astype(rasterio.float32), 1)
        
    # scale wrapped interferogram (0 to 255)
    scaled_path = wrapped_path/f'{wrp_path.stem.split("_unscaled")[0]}_scaled.tif'
    !gdal_translate -of GTiff -scale -ot BYTE $wrp_path $scaled_path
    wrp_path.unlink()
    
    # add color ramp
    colorize_wrapped_insar(scaled_path)
    
    # convert to 3-band rgb
    three_band_path = wrapped_path/f'{scaled_path.stem.split("_scaled")[0]}.tif'
    !gdal_translate -of GTiff -expand rgb $scaled_path $three_band_path
    scaled_path.unlink()

**Save the temporal coherence geotiff**

In [ ]:
!save_gdal.py $mint_path/temporalCoherence.h5 --of GTIFF -o $geotiff_path/TemporalCoherence.tif

**Save the average spatial coherence geotiff**

In [ ]:
!save_gdal.py $mint_path/avgSpatialCoh.h5 --of GTIFF -o $geotiff_path/avgSpatialCoh.tif

**Save the velocity geotiff**

In [ ]:
velocity_name = "velocity"
if correct_tropo:
    velocity_name = f'{velocity_name}ERA5'
vel_h5 = mint_path/f'{velocity_name}.h5'
vel_tiff = geotiff_path/f'{velocity_name}.tif'
!save_gdal.py $vel_h5 --of GTIFF -o $vel_tiff

# Reference material

- Mintpy reference: *Yunjun, Z., H. Fattahi, F. Amelung (2019), Small baseline InSAR time series analysis: unwrapping error correction and noise reduction, preprint doi:[10.31223/osf.io/9sz6m](https://eartharxiv.org/9sz6m/).*

- University of Miami online time-series viewer: https://insarmaps.miami.edu/

- Mintpy Github repository: https://github.com/insarlab/MintPy

*MintPy_Time_Series_From_Prepared_Data_Stack.ipynb - Version 0.4.0 - June 2023*

*Version Changes*

- *Add option to skip data load and template creation on repeat runs*
- *Adjust colorize_wrapped_insar function to better match "jet"*
- *Use cumulative displacement for transect selection plot*
- *Add GPS site and metadata viewer*